In [200]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word.lower():
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True
    
    def search(self, word):
        node = self.root
        for char in word.lower():
            if char not in node.children:
                return False
            node = node.children[char]
        return node.is_end_of_word

    def starts_with(self, prefix):
        node = self.root
        for char in prefix.lower():
            if char not in node.children:
                return False
            node = node.children[char]
        return True

def load_words_in_trie():
    trie = Trie()
    with open('words.txt', 'r') as file:
        for line in file:
            trie.insert(line.strip())
    return trie

valid_dict = load_words_in_trie()


In [238]:
from collections import Counter
from collections import defaultdict
import pyfiglet

class Board:
    def __init__(self, letters, trie_dict=valid_dict):

        self.grid_size = int(len(letters) ** 0.5)
        if self.grid_size ** 2 != len(letters):
            print("!! The letters don't form a square grid. Lenght: ", len(letters))

        self.letters = letters
        self.all_words_trie = trie_dict
        self.valid_words = set() 

    def __str__(self): 
        return self._format_board(self.letters)
    
    @staticmethod
    def _format_board(letters):
        dim = int(len(letters) ** 0.5)
        if dim ** 2 != len(letters):
            print(f"Board with length {len(letters)} is not a square.")
            return
        board_string = ''
        for i in range(0, len(letters), dim):
            row = '  '.join(letters[i:i+dim])
            board_string += f"{row}\n"
        return board_string.upper()
    
    @staticmethod
    def _get_board_dimensions(letters):
        dim = int(len(letters) ** 0.5)
        if dim ** 2 != len(letters):
            raise ValueError(f"Board with length {len(letters)} is not a square.")
        return dim

    @staticmethod
    def print_board(letters, font="ntgreek"):
        n = Board._get_board_dimensions(letters)
        grid_string = Board._format_board(letters)
        # print(pyfiglet.figlet_format(grid_string, font=font))
        print(Board._format_board(letters))
                
    def get_letter_neighbors(self):
        """ return a dict of dicts, where dict[c1][c2] gives the number of times char c2 appeared next to c1"""
        alpha = "abcdefghijklmnopqrstuvwxyz"
        letter_neighbors = {char: {char: 0 for char in alpha} for char in alpha}
        for index, char in enumerate(self.letters):
            neighbors_indices = self._get_neighbors_from_char_index(index)
            for i in neighbors_indices:
                letter_neighbors[char][self.letters[i]] += 1
        return letter_neighbors

    def _to_1d_index(self, row, col):
        return (self.grid_size * row) + col


    def _get_neighbors_from_char_index(self, index):
        neighbor_indices = []

        # convert index to 0-indexed row/col
        i, j = divmod(index, self.grid_size)

        # Directions: NSEW + diagonals
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1),    # NSEW
                      (-1, -1), (-1, 1), (1, -1), (1, 1)]  # Diagonals

        for di, dj in directions:
            ni, nj = i + di, j + dj
            # check boundaries
            if 0 <= ni < self.grid_size and 0 <= nj < self.grid_size:
                neighbor_indices.append(self._to_1d_index(ni, nj))

        return neighbor_indices


    def _get_unused_neighbors(self, index, used):
        return [n for n in self._get_neighbors_from_char_index(index) if not used[n]]
        

    def _find_all_valid_words(self):

        def dfs(fragment, prev_char_index, used):
            if len(fragment) >= 3 and self.all_words_trie.search(fragment):
                self.valid_words.add(fragment)
            if not self.all_words_trie.starts_with(fragment):
                return 
            unused_neighbors = self._get_unused_neighbors(prev_char_index, used)
            for char_index in unused_neighbors:
                new_fragment = fragment + self.letters[char_index]
                updated_used_list = used[:]
                updated_used_list[char_index] = 1
                dfs(new_fragment, char_index, updated_used_list)

        # array tracking whether a letter index has been used or not
        used = [0 for char in self.letters]

        # search for wards starting with each character
        for index, char in enumerate(self.letters):
            updated_used = used[:]
            updated_used[index] = 1
            dfs(char, index, updated_used)

        
    def print_valid_words(self):
        if len(self.valid_words) == 0:
            self._find_all_valid_words()
        sorted_strings = sorted(self.valid_words, key=lambda s: (-len(s), s))

        grouped = defaultdict(list)
        for word in self.valid_words:
            grouped[len(word)].append(word)
        
        # Sort by length in descending order and convert to list of lists
        result = [grouped[length] for length in sorted(grouped.keys(), reverse=True)]
      
        # Output the result
        for group in result:
            print(group)
            
        # print(sorted_strings)


    def print_word_lengths(self):
        if len(self.valid_words) == 0:
            self._find_all_valid_words()
        length_counts = Counter(len(word) for word in self.valid_words)
        # Sort by length in descending order
        for length, count in sorted(length_counts.items(), key=lambda x: x[0], reverse=True):
            print(f"Words with {length} characters: {count}")
    

    def print_word_scores(self):
        if len(self.valid_words) == 0:
            self._find_all_valid_words()
        def points(length):
            if length == 3: 
                return 100
            elif length == 4: return 400
            elif length == 5: return 800
            else: return 200 + (400 * (length - 3))
        length_counts = Counter(len(word) for word in self.valid_words)
        # Sort by length in descending order
        for length, count in sorted(length_counts.items(), key=lambda x: x[0], reverse=True):
            print(f"{points(length)}: {count}")
        
    
    def get(self, row, col):
        """ get letter at the 1-indexed row and column """
        if row < 1 or row > 4 or col < 1 or col > 4:
            return -1
        index = (4 * (row - 1)) + (col - 1)
        return self.letters[index]




In [239]:
import statistics 

# Load wordhunt boards from file
real_boards = []
with open('wordhunt_boards.txt', 'r') as file:
    for line in file:
        real_boards.append(line.strip())

# generate weights
def compute_letter_frequencies(word_list):
    letter_counts = Counter(char for word in word_list for char in word)
    sorted_letter_counts = dict(sorted(letter_counts.items()))
    # Normalize
    total_letters = sum(len(word) for word in word_list)
    for key in sorted_letter_counts:
        sorted_letter_counts[key] /= total_letters
    return sorted_letter_counts

def get_index_letter_frequencies(word_list, index):
    letter_counts = Counter(word[index] for word in word_list)
    for char in 'abcdefghijklmnopqrstuvwxyz':
        if char not in letter_counts:
            letter_counts[char] = 0
    sorted_letter_counts = dict(sorted(letter_counts.items()))
    # Normalize
    # total_letters = sum(len(word) for word in word_list)
    # for key in sorted_letter_counts:
    #     sorted_letter_counts[key] /= total_letters
    return sorted_letter_counts

# Get frequencies of letters occuring at a specific index of the board
index_frequencies = []
for i in range(16):
    index_frequencies.append(get_index_letter_frequencies(real_boards, i))
print("\tmax\tmin\tavg\tmed\tstdev")
for key in index_frequencies[0]:
    f = [freq[key] for freq in index_frequencies]
    print(f"{key}\t{max(f)}\t{min(f)}\t{statistics.mean(f)}\t{statistics.median(f)}\t{statistics.stdev(f)}")

# gamepigeon_weights = compute_letter_frequencies(real_boards)
# print(gamepigeon_weights)


	max	min	avg	med	stdev
a	8	0	3.9375	3.5	1.9822125684867067
b	2	0	0.8125	1.0	0.8341662504161466
c	3	0	1.125	1.0	1.0878112581387147
d	4	0	2.1875	2.0	1.5152007567755943
e	10	1	5.125	4.0	2.6551836094703507
f	2	0	0.75	0.0	0.9309493362512627
g	3	0	1.3125	1.0	1.138346754435279
h	7	0	3.625	3.5	2.0289570391377603
i	10	1	3.5625	3.0	2.337199178504049
j	1	0	0.125	0.0	0.3415650255319866
k	3	0	0.75	1.0	0.8563488385776752
l	7	0	2.125	1.0	2.1252450839060106
m	4	0	1.125	1.0	1.1474609652039003
n	10	0	3.75	4.0	2.4083189157584592
o	8	1	3.5625	3.0	2.1899391163530857
p	3	0	1.125	1.0	0.9574271077563381
q	1	0	0.0625	0.0	0.25
r	6	0	3.25	3.5	1.7320508075688772
s	6	1	3.3125	4.0	1.5370426148939398
t	7	0	3.3125	3.0	1.9224550276491081
u	5	0	1.75	1.0	1.6532795690182993
v	2	0	0.8125	1.0	0.8341662504161466
w	4	0	1.375	1.0	1.408308678285174
x	1	0	0.0625	0.0	0.25
y	2	0	1.0625	1.0	0.7719024117939607
z	0	0	0	0.0	0.0


In [240]:

# Get frequencies of letters appearing next to other letters
alpha = "abcdefghijklmnopqrstuvwxyz"
agg_neighbor_freq = {char: {char: 0 for char in alpha} for char in alpha}
for board in real_boards:
    nf = Board(board).get_letter_neighbors()
    for char in nf:
        for nchar in nf[char]:
            agg_neighbor_freq[char][nchar] += nf[char][nchar]
print("\t", "\t".join(c for c in alpha))
for char in agg_neighbor_freq:
    print(char+":\t", "\t".join(str(agg_neighbor_freq[char][nchar]) for nchar in alpha))

	 a	b	c	d	e	f	g	h	i	j	k	l	m	n	o	p	q	r	s	t	u	v	w	x	y	z
a:	 24	5	5	10	36	6	7	22	25	0	4	17	6	29	13	8	0	31	20	17	11	5	12	0	7	0
b:	 5	0	1	4	5	1	1	3	4	0	1	3	0	7	3	1	0	2	6	7	0	3	2	0	2	0
c:	 5	1	2	3	9	1	2	9	12	2	0	5	3	8	11	1	0	4	4	4	3	0	3	0	5	0
d:	 10	4	3	8	20	2	8	15	12	1	1	10	5	14	13	3	2	13	12	9	3	4	9	0	4	0
e:	 36	5	9	20	24	6	14	40	28	0	9	17	13	32	28	7	0	31	41	29	14	8	11	2	8	0
f:	 6	1	1	2	6	0	0	5	7	0	1	2	1	3	7	1	0	8	6	6	1	1	3	0	0	0
g:	 7	1	2	8	14	0	4	1	9	0	2	1	5	10	8	4	1	11	7	4	1	2	4	0	1	0
h:	 22	3	9	15	40	5	1	10	18	1	3	14	7	28	28	7	0	21	17	15	11	4	11	1	7	0
i:	 25	4	12	12	28	7	9	18	10	0	6	13	10	21	20	7	1	19	26	25	12	8	7	2	7	0
j:	 0	0	2	1	0	0	0	1	0	0	1	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0
k:	 4	1	0	1	9	1	2	3	6	1	0	3	1	7	4	0	0	3	5	4	3	0	1	0	1	0
l:	 17	3	5	10	17	2	1	14	13	0	3	12	6	12	25	2	0	9	15	9	4	5	2	0	4	0
m:	 6	0	3	5	13	1	5	7	10	0	1	6	0	4	6	4	0	6	8	7	5	2	3	0	4	0
n:	 29	7	8	14	32	3	10	28	21	0	7	12	4	6	19	11	0	26	22	22	10	1	12	1	8	0
o:	 13	3	11	13	28	7	8	28	20	0	4	25	6	19	12	7	0	19	20	25	9	6	3	1	6	0


In [241]:
import random

def generate_weighted_string(n, char_weights):
    characters, weights = zip(*char_weights.items())
    board = ''.join(random.choices(characters, weights=weights, k=n))
    return board

# manual weights
char_weights = {
    'a': 9, 'b': 3, 'c': 3, 'd': 7, 'e': 9,
    'f': 3, 'g': 3, 'h': 9, 'i': 9, 'j': 1,
    'k': 2, 'l': 3, 'm': 3, 'n': 9, 'o': 9,
    'p': 3, 'q': 1, 'r': 9, 's': 8, 't': 9,
    'u': 6, 'v': 2, 'w': 3, 'x': 1, 'y': 3, 'z': 1 
}
n = 16  # Length of the desired string

# Some random boards using the same letter frequencies as gamepigeon
for i in range(5):
  weighted_string = generate_weighted_string(n, gamepigeon_weights)
  board = Board(weighted_string)
  print(board)
  board.print_valid_words()
  board.print_word_scores()
  print("\n\n")




W  N  I  S
H  R  S  L
L  A  S  R
S  A  C  D

['harissas']
['harissa', 'ascaris', 'scalars', 'lascars', 'rascals']
['lassis', 'lascar', 'rascal', 'scalar', 'sisals', 'cassis', 'salsas', 'sarins', 'laaris']
['harls', 'laris', 'acari', 'liras', 'harns', 'sasin', 'salsa', 'arils', 'casas', 'carns', 'scala', 'saris', 'nirls', 'scars', 'sarin', 'larns', 'carls', 'laari', 'sisal', 'lassi', 'arsis']
['nils', 'cars', 'aril', 'carl', 'siss', 'asar', 'casa', 'lars', 'scar', 'rins', 'caas', 'sari', 'lacs', 'carn', 'nirl', 'lari', 'sacs', 'lira', 'lins', 'hass', 'aris', 'aals', 'sass', 'sars', 'cals', 'harl', 'sals', 'alas', 'larn', 'lass', 'sins', 'harn', 'sris', 'sirs', 'raca', 'sinh', 'alar']
['lah', 'lac', 'lar', 'caa', 'sar', 'car', 'sri', 'nil', 'ars', 'aal', 'sac', 'lis', 'nis', 'wha', 'ass', 'cal', 'sir', 'lin', 'rah', 'ras', 'rin', 'aas', 'sis', 'sin', 'aah', 'ins', 'has', 'ala', 'sal', 'als', 'las']
2200: 1
1800: 5
1400: 9
800: 21
400: 37
100: 31



S  O  I  I
C  A  N  T
S  O  J  I
N  A  

In [242]:
# Real boards from our last 50 games
for board in real_boards:
  board = Board(real_boards[10])
  print(board)
  board.print_valid_words()
  board.print_word_scores()
  # board.print_word_lengths()


P  T  Y  T
M  N  I  N
W  S  A  H
A  F  R  U

['safranin']
['uranism', 'uranins']
['sanity', 'nanism', 'ptisan', 'fainty', 'uranin']
['antis', 'rainy', 'hinau', 'faint', 'snarf', 'tinty', 'tiars', 'tians', 'fawny', 'hains', 'hiant', 'tiyns', 'fawns', 'tinas', 'saint', 'hansa', 'ranis', 'rains', 'haint', 'afars', 'ahint', 'fains']
['rant', 'sawn', 'mnas', 'isna', 'tint', 'tian', 'fawn', 'hint', 'asar', 'fain', 'rais', 'yins', 'tins', 'snar', 'faws', 'arfs', 'snit', 'rait', 'anti', 'anis', 'ursa', 'rani', 'ansa', 'ains', 'nain', 'fras', 'wasm', 'tiny', 'sain', 'tiyn', 'tiar', 'awns', 'faur', 'rain', 'awny', 'nans', 'ahis', 'fars', 'afar', 'tyin', 'frau', 'fans', 'hisn', 'hins', 'sinh', 'sant', 'hain', 'hant', 'tina']
['nas', 'san', 'ant', 'mna', 'nan', 'any', 'sar', 'ans', 'sai', 'hit', 'rai', 'ait', 'tit', 'ais', 'ars', 'tis', 'was', 'nit', 'ahi', 'ain', 'yin', 'han', 'ani', 'nah', 'fra', 'far', 'nis', 'his', 'ran', 'sny', 'fas', 'arf', 'rah', 'ras', 'saw', 'fah', 'ism', 'awn', 'sin', 'h